Importação das bibliotecas

In [1]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
import nltk
import unidecode

from nltk.tokenize import wordpunct_tokenize
from string import punctuation
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.dummy import DummyClassifier
from sklearn.multiclass import OneVsRestClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.naive_bayes import MultinomialNB
from sklearn.tree import DecisionTreeClassifier
from sklearn.svm import SVC
from sklearn.model_selection import GridSearchCV

Importando os dados

In [2]:
treino = pd.read_csv('../dados/dados_treino.csv')
teste = pd.read_csv('../dados/dados_teste.csv')

In [3]:
treino.dropna(inplace = True)
teste.dropna(inplace = True)

Utilizando transformação TF-IDF nos dados

In [30]:
tfidf = TfidfVectorizer(lowercase= True, max_features= 3000, max_df = 0.80)

In [31]:
tfidf.fit(treino['texto'].astype('U'))

TfidfVectorizer(max_df=0.8, max_features=3000)

In [32]:
perguntas_treino = tfidf.transform(treino['texto'].astype('U'))

In [33]:
perguntas_teste = tfidf.transform(teste['texto'].astype('U'))

Construindo um modelo de base para comparação (DummyClassifier)

In [34]:
dummy = DummyClassifier(strategy='most_frequent')
dummy.fit(perguntas_treino, treino['Categoria'])

DummyClassifier(strategy='most_frequent')

In [35]:
resultado_treino = dummy.score(perguntas_treino, treino['Categoria'])
resultado_teste = dummy.score(perguntas_teste, teste['Categoria'])
resultados = pd.DataFrame({'Modelo':['DummyClassifier'], 'Acurácia de treino': [resultado_treino], 'Acurácia de teste': [resultado_teste]})

In [36]:
resultados

,Modelo,Acurácia de treino,Acurácia de teste
0,DummyClassifier,0.142857,0.143266


Construindo um modelo de Regressão Logística One Vs Rest

In [37]:
treino_LR = pd.get_dummies(treino, columns = ['Categoria'])
teste_LR = pd.get_dummies(teste, columns = ['Categoria'])

lista_zip_treino = list(zip(treino_LR['Categoria_Data Science'],
                            treino_LR['Categoria_DevOps'],
                            treino_LR['Categoria_Front-End'],
                            treino_LR['Categoria_Inovação & Gestão'],
                            treino_LR['Categoria_Mobile'],
                            treino_LR['Categoria_Programação'],
                            treino_LR['Categoria_UX & Design']
))

lista_zip_teste = list(zip(teste_LR['Categoria_Data Science'],
                            teste_LR['Categoria_DevOps'],
                            teste_LR['Categoria_Front-End'],
                            teste_LR['Categoria_Inovação & Gestão'],
                            teste_LR['Categoria_Mobile'],
                            teste_LR['Categoria_Programação'],
                            teste_LR['Categoria_UX & Design']
))

treino_LR['target'] = lista_zip_treino
teste_LR['target'] = lista_zip_teste

target_array_treino = np.asarray(list(treino_LR['target']))
target_array_teste = np.asarray(list(teste_LR['target']))

regressao_logistica = LogisticRegression(max_iter=200)
classificador_onerest = OneVsRestClassifier(regressao_logistica)
classificador_onerest.fit(perguntas_treino, target_array_treino)


OneVsRestClassifier(estimator=LogisticRegression(max_iter=200))

In [38]:
resultado_treino = classificador_onerest.score(perguntas_treino, target_array_treino)
resultado_teste = classificador_onerest.score(perguntas_teste, target_array_teste)
resultados = resultados.append({'Modelo':'LogisticRegression', 'Acurácia de treino': resultado_treino, 'Acurácia de teste': resultado_teste}, ignore_index = True)

In [39]:
resultados

,Modelo,Acurácia de treino,Acurácia de teste
0,DummyClassifier,0.142857,0.143266
1,LogisticRegression,0.533607,0.496848


Construindo um modelo Naive Bayes Multinomial

In [40]:
naivieb = MultinomialNB()
naivieb.fit(perguntas_treino, treino['Categoria'])

MultinomialNB()

In [41]:
resultado_treino = naivieb.score(perguntas_treino, treino['Categoria'])
resultado_teste = naivieb.score(perguntas_teste, teste['Categoria'])
resultados = resultados.append({'Modelo':'MultinomialNB', 'Acurácia de treino': resultado_treino, 'Acurácia de teste': resultado_teste}, ignore_index = True)

In [42]:
resultados

,Modelo,Acurácia de treino,Acurácia de teste
0,DummyClassifier,0.142857,0.143266
1,LogisticRegression,0.533607,0.496848
2,MultinomialNB,0.690964,0.664040


Construindo um modelo de Árvore de Decisão

In [43]:
dtree = DecisionTreeClassifier()
dtree.fit(perguntas_treino, treino['Categoria'])

DecisionTreeClassifier()

In [44]:
resultado_treino = dtree.score(perguntas_treino, treino['Categoria'])
resultado_teste = dtree.score(perguntas_teste, teste['Categoria'])
resultados = resultados.append({'Modelo':'DecisionTreeClassifier', 'Acurácia de treino': resultado_treino, 'Acurácia de teste': resultado_teste}, ignore_index = True)

In [45]:
resultados

,Modelo,Acurácia de treino,Acurácia de teste
0,DummyClassifier,0.142857,0.143266
1,LogisticRegression,0.533607,0.496848
2,MultinomialNB,0.690964,0.664040
3,DecisionTreeClassifier,0.991214,0.507593


Construindo um modelo SVM

In [46]:
svm = SVC(kernel = 'linear', C = 1)
svm.fit(perguntas_treino, treino['Categoria'])

SVC(C=1, kernel='linear')

In [47]:
resultado_treino = svm.score(perguntas_treino, treino['Categoria'])
resultado_teste = svm.score(perguntas_teste, teste['Categoria'])
resultados = resultados.append({'Modelo':'SVM', 'Acurácia de treino': resultado_treino, 'Acurácia de teste': resultado_teste}, ignore_index = True)

In [48]:
resultados

,Modelo,Acurácia de treino,Acurácia de teste
0,DummyClassifier,0.142857,0.143266
1,LogisticRegression,0.533607,0.496848
2,MultinomialNB,0.690964,0.664040
3,DecisionTreeClassifier,0.991214,0.507593
4,SVM,0.800107,0.708453


Tratando melhor os dados textuais

In [4]:
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\Admin\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [5]:
palavras_irrelevantes = nltk.corpus.stopwords.words("portuguese")
pontuacao = [ponto for ponto in punctuation]

In [6]:
remocoes = palavras_irrelevantes + pontuacao

Código utilizado para remover stopwords, acentuação, pontuação e colocar todo o texto em minúsculo

In [7]:
textos_tratados = []
for texto in treino['texto']:
    novo_texto = []
    texto_sem_acento = unidecode.unidecode(texto.lower())
    palavras_texto = wordpunct_tokenize(texto_sem_acento)
    for palavra in palavras_texto:
        if palavra not in remocoes:
            novo_texto.append(palavra)
    textos_tratados.append(' '.join(novo_texto))

In [8]:
treino['texto_tratado'] = textos_tratados

In [9]:
textos_tratados = []
for texto in teste['texto']:
    novo_texto = []
    texto_sem_acento = unidecode.unidecode(texto.lower())
    palavras_texto = wordpunct_tokenize(texto_sem_acento)
    for palavra in palavras_texto:
        if palavra not in remocoes:
            novo_texto.append(palavra)
    textos_tratados.append(' '.join(novo_texto))

In [10]:
teste['texto_tratado'] = textos_tratados

In [11]:
tfidf = TfidfVectorizer(max_features= 3000, max_df = 0.80)
tfidf.fit(treino['texto_tratado'].astype('U'))
perguntas_treino = tfidf.transform(treino['texto_tratado'].astype('U'))
perguntas_teste = tfidf.transform(teste['texto_tratado'].astype('U'))

Treinando novamente os modelos após tratamento de dados

In [99]:
dummy = DummyClassifier(strategy='most_frequent')
dummy.fit(perguntas_treino, treino['Categoria'])
resultado_treino = dummy.score(perguntas_treino, treino['Categoria'])
resultado_teste = dummy.score(perguntas_teste, teste['Categoria'])
resultados = pd.DataFrame({'Modelo':['DummyClassifier'], 'Acurácia de treino': [resultado_treino], 'Acurácia de teste': [resultado_teste]})

In [100]:
treino_LR = pd.get_dummies(treino, columns = ['Categoria'])
teste_LR = pd.get_dummies(teste, columns = ['Categoria'])

lista_zip_treino = list(zip(treino_LR['Categoria_Data Science'],
                            treino_LR['Categoria_DevOps'],
                            treino_LR['Categoria_Front-End'],
                            treino_LR['Categoria_Inovação & Gestão'],
                            treino_LR['Categoria_Mobile'],
                            treino_LR['Categoria_Programação'],
                            treino_LR['Categoria_UX & Design']
))

lista_zip_teste = list(zip(teste_LR['Categoria_Data Science'],
                            teste_LR['Categoria_DevOps'],
                            teste_LR['Categoria_Front-End'],
                            teste_LR['Categoria_Inovação & Gestão'],
                            teste_LR['Categoria_Mobile'],
                            teste_LR['Categoria_Programação'],
                            teste_LR['Categoria_UX & Design']
))

treino_LR['target'] = lista_zip_treino
teste_LR['target'] = lista_zip_teste

target_array_treino = np.asarray(list(treino_LR['target']))
target_array_teste = np.asarray(list(teste_LR['target']))

regressao_logistica = LogisticRegression(max_iter=200)
classificador_onerest = OneVsRestClassifier(regressao_logistica)
classificador_onerest.fit(perguntas_treino, target_array_treino)
resultado_treino = classificador_onerest.score(perguntas_treino, target_array_treino)
resultado_teste = classificador_onerest.score(perguntas_teste, target_array_teste)
resultados = resultados.append({'Modelo':'LogisticRegression', 'Acurácia de treino': resultado_treino, 'Acurácia de teste': resultado_teste}, ignore_index = True)

In [101]:
naivieb = MultinomialNB()
naivieb.fit(perguntas_treino, treino['Categoria'])
resultado_treino = naivieb.score(perguntas_treino, treino['Categoria'])
resultado_teste = naivieb.score(perguntas_teste, teste['Categoria'])
resultados = resultados.append({'Modelo':'MultinomialNB', 'Acurácia de treino': resultado_treino, 'Acurácia de teste': resultado_teste}, ignore_index = True)

In [102]:
dtree = DecisionTreeClassifier()
dtree.fit(perguntas_treino, treino['Categoria'])
resultado_treino = dtree.score(perguntas_treino, treino['Categoria'])
resultado_teste = dtree.score(perguntas_teste, teste['Categoria'])
resultados = resultados.append({'Modelo':'DecisionTreeClassifier', 'Acurácia de treino': resultado_treino, 'Acurácia de teste': resultado_teste}, ignore_index = True)

In [103]:
svm = SVC(kernel = 'linear', C = 1)
svm.fit(perguntas_treino, treino['Categoria'])
resultado_treino = svm.score(perguntas_treino, treino['Categoria'])
resultado_teste = svm.score(perguntas_teste, teste['Categoria'])
resultados = resultados.append({'Modelo':'SVM', 'Acurácia de treino': resultado_treino, 'Acurácia de teste': resultado_teste}, ignore_index = True)

In [104]:
resultados

,Modelo,Acurácia de treino,Acurácia de teste
0,DummyClassifier,0.143237,0.140880
1,LogisticRegression,0.552587,0.513194
2,MultinomialNB,0.712018,0.678731
3,DecisionTreeClassifier,0.994183,0.548666
4,SVM,0.811771,0.716799


Realizando o tuning de hiperparâmetros para o modelo SVM, que se saiu melhor nos resultados de treinamento

In [130]:
parametros = {'C': [0.1, 1, 10, 100],
              'gamma': ['scale', 1, 0.1, 0.01, 0.001],
              'kernel': ['rbf', 'linear']}

svm_gs = GridSearchCV(SVC(), parametros, verbose = 3)
svm_gs.fit(perguntas_treino, treino['Categoria'])

Fitting 5 folds for each of 40 candidates, totalling 200 fits
[CV] C=0.1, gamma=scale, kernel=rbf ..................................


[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.


[CV] ...... C=0.1, gamma=scale, kernel=rbf, score=0.623, total= 5.0min
[CV] C=0.1, gamma=scale, kernel=rbf ..................................


[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:  5.0min remaining:    0.0s


[CV] ...... C=0.1, gamma=scale, kernel=rbf, score=0.610, total= 4.9min
[CV] C=0.1, gamma=scale, kernel=rbf ..................................


[Parallel(n_jobs=1)]: Done   2 out of   2 | elapsed:  9.9min remaining:    0.0s


[CV] ...... C=0.1, gamma=scale, kernel=rbf, score=0.612, total= 4.9min
[CV] C=0.1, gamma=scale, kernel=rbf ..................................
[CV] ...... C=0.1, gamma=scale, kernel=rbf, score=0.618, total= 4.8min
[CV] C=0.1, gamma=scale, kernel=rbf ..................................
[CV] ...... C=0.1, gamma=scale, kernel=rbf, score=0.613, total= 5.0min
[CV] C=0.1, gamma=scale, kernel=linear ...............................
[CV] ... C=0.1, gamma=scale, kernel=linear, score=0.677, total= 3.5min
[CV] C=0.1, gamma=scale, kernel=linear ...............................
[CV] ... C=0.1, gamma=scale, kernel=linear, score=0.671, total= 3.5min
[CV] C=0.1, gamma=scale, kernel=linear ...............................
[CV] ... C=0.1, gamma=scale, kernel=linear, score=0.676, total= 3.4min
[CV] C=0.1, gamma=scale, kernel=linear ...............................
[CV] ... C=0.1, gamma=scale, kernel=linear, score=0.679, total= 3.4min
[CV] C=0.1, gamma=scale, kernel=linear ...............................
[CV] .

[Parallel(n_jobs=1)]: Done 200 out of 200 | elapsed: 2070.2min finished


GridSearchCV(estimator=SVC(),
             param_grid={'C': [0.1, 1, 10, 100],
                         'gamma': ['scale', 1, 0.1, 0.01, 0.001],
                         'kernel': ['rbf', 'linear']},
             verbose=3)

In [131]:
print(svm_gs.best_params_)
print(svm_gs.best_score_)

{'C': 10, 'gamma': 'scale', 'kernel': 'rbf'}
0.7194513098767767


In [25]:
svm = SVC(kernel = 'rbf', C = 10, gamma='scale')
svm.fit(perguntas_treino, treino['Categoria'])
resultado_treino = svm.score(perguntas_treino, treino['Categoria'])
resultado_teste = svm.score(perguntas_teste, teste['Categoria'])
resultados = pd.DataFrame({'Modelo':['SVM'], 'Acurácia de treino': [resultado_treino], 'Acurácia de teste': [resultado_teste]})

In [26]:
resultados

,Modelo,Acurácia de treino,Acurácia de teste
0,SVM,0.994075,0.728479


In [12]:
svm = SVC(kernel = 'linear', C = 1, gamma='scale')
svm.fit(perguntas_treino, treino['Categoria'])
resultado_treino = svm.score(perguntas_treino, treino['Categoria'])
resultado_teste = svm.score(perguntas_teste, teste['Categoria'])
resultados = pd.DataFrame({'Modelo':['SVM'], 'Acurácia de treino': [resultado_treino], 'Acurácia de teste': [resultado_teste]})

In [13]:
resultados

,Modelo,Acurácia de treino,Acurácia de teste
0,SVM,0.811771,0.716799


Aplicando o modelo nos dados de tópicos da seção Off-topic

In [14]:
topicos = pd.read_csv('../dados/Off-topic com textos.csv')

In [16]:
topicos.dropna(inplace = True)

In [17]:
textos_tratados = []
for texto in topicos['texto']:
    novo_texto = []
    texto_sem_acento = unidecode.unidecode(texto.lower())
    palavras_texto = wordpunct_tokenize(texto_sem_acento)
    for palavra in palavras_texto:
        if palavra not in remocoes:
            novo_texto.append(palavra)
    textos_tratados.append(' '.join(novo_texto))

In [18]:
topicos['texto_tratado'] = textos_tratados
textos_off_topic = tfidf.transform(topicos['texto_tratado'].astype('U'))

In [19]:
categorias_previstas = svm.predict(textos_off_topic)

In [20]:
categorias_previstas

array(['Front-End', 'Front-End', 'DevOps', ..., 'Mobile', 'Front-End',
       'Mobile'], dtype=object)

In [21]:
topicos['categoria'] = categorias_previstas

In [22]:
topicos[['titulo', 'links', 'categoria']].to_excel('../dados/Off-topic categorizado.xlsx', index = False)